In [15]:
import os
import subprocess
import sys
from tkinter import filedialog
import tkinter as tk
import whisper
import ollama
# necessary imports for the application


In [16]:
# functions used with the models for extracting the text and post-processing it
model = whisper.load_model("small", device="cpu")
def transcribe_audio(selected_file: str):
    try:
        label_file_explorer.config(text="Processing...", foreground="orange")
        window.update()

        file_dir = os.path.dirname(selected_file)
        file_name = os.path.splitext(os.path.basename(selected_file))[0]

        result = model.transcribe(selected_file)
        text = extract_medical_context(result['text'])
        parts = text.split(",")
        final = "\n".join(parts)
        print(final)

        tex_file_path = os.path.join(file_dir, f"{file_name}.tex")

        with open(tex_file_path, 'w', encoding="utf-8") as f:
            f.write('\\documentclass{article}\n')
            f.write('\\usepackage[english,romanian]{babel}\n')
            f.write('\\usepackage{graphicx}\n')
            f.write('\\usepackage[margin=2cm]{geometry}\n')
            f.write('\\usepackage{parskip}\n')
            f.write('\\begin{document}\n')

            f.write('\\begin{center}\n')
            f.write('\\includegraphics[width=3cm]{/home/raulpuscas/ai/proj/logo.jpg}\\\\[0.5cm]\n')
            f.write('{\\LARGE\\bfseries Fisa medicala}\\\\[0.2cm]\n')
            f.write('{\\normalsize Dr. Echipa AI-VROS}\\\\[0.3cm]\n')
            f.write('\\end{center}\n')

            for line in parts:
                f.write(line.strip() + "\\\\\n")

            f.write('\\vspace{0.8cm}\n')
            f.write('\\begin{center}\n')
            f.write('\\small Multumim pentru increderea acordata in clinica LAB AI 2025!\n')
            f.write('\\end{center}\n')
            f.write('\\end{document}\n')

        if sys.platform == 'linux':
            pid = os.fork()
            if pid == 0:  
                try:
                    subprocess.run(
                        ['pdflatex', '-interaction=nonstopmode', tex_file_path],
                        check=True,
                        cwd=file_dir
                    )
                    os._exit(0)  
                except Exception as e:
                    print(f"Child process error: {e}")
                    os._exit(1)
        else:
            subprocess.run([
                r'C:\Users\ioana\AppData\Local\Programs\MiKTeX\miktex\bin\x64\pdflatex.exe',
                tex_file_path
            ], check=True, cwd=file_dir)

        label_file_explorer.config(text=f"Success! PDF generated: {file_name}.pdf", foreground="green")

        pdf_path = os.path.join(file_dir, f"{file_name}.pdf")

        if sys.platform == 'linux':
            if 'pid' in locals():
                os.waitpid(pid, 0)

        os.startfile(pdf_path) if sys.platform == 'win32' else subprocess.run(['xdg-open', pdf_path])

    except Exception as e:
        label_file_explorer.config(text=f"Error: {str(e)}", foreground="red")
        print(f"An error occurred: {e}")

def extract_medical_context(text: str) -> str:
    """
    Extracts raw medical information from text without any formatting or commentary.
    Returns only the medical terms/phrases as they appear in the original text.
    """
    prompt = (
        "Extract only the exact medical terms and phrases from this text. "
        "Include only:\n"
        "- Diagnoses\n"
        "- Symptoms\n"
        "- Medications\n"
        "- Measurements\n"
        "- Anatomical references\n"
        "- Procedures\n\n"
        "RULES:\n"
        "1. Return ONLY the medical terms/phrases exactly as they appear\n"
        "2. No formatting, bullet points, or categories\n"
        "3. No explanations or translations\n"
        "4. Separate multiple terms with newlines\n"
        "5. Preserve original wording and units\n\n"
        f"TEXT TO PROCESS:\n{text}\n\n"
        "EXTRACTED MEDICAL TERMS:"
    )

    response = ollama.chat(
        model='llama3-chatqa',
        messages=[{
            'role': 'user',
            'content': prompt
        }],
        options={'temperature': 0.1}  # Makes output more factual
    )
    
    # Post-processing to remove any residual formatting
    result = response['message']['content']
    return result


In [17]:
# the function used by u
def browseFiles():
    global selected_file
    selected_file = filedialog.askopenfilename(initialdir="/",
                                               title="Select a File",
                                               filetypes=(("Audio files", "*.wav;*.m4a;*.flac;*.mp3;*.ogg"), ("All files", "*.*")))
    if selected_file:
        label_file_explorer.configure(text="File Opened: " + selected_file)
        print("Selected file path:", selected_file)
        transcribe_audio(selected_file)



In [ ]:
# main application
import tkinter as tk
from tkinter import ttk



def exit():
    window.destroy()

window = tk.Tk()
window.title("File Explorer")
window.geometry("600x400")
window.config(background="#f0f0f0")

style = ttk.Style()
style.theme_use('clam')


style.configure('TLabel', background="#f0f0f0", font=('Helvetica', 12))
style.configure('TButton', font=('Helvetica', 10), padding=6)
style.configure('Title.TLabel', font=('Helvetica', 14, 'bold'))

header_frame = ttk.Frame(window, style='TFrame')
header_frame.grid(column=0, row=0, columnspan=2, pady=10, padx=10, sticky="ew")

label_title = ttk.Label(header_frame,
                       text="File Explorer",
                       style='Title.TLabel',
                       foreground="#2c3e50")
label_title.pack()

content_frame = ttk.Frame(window, style='TFrame')
content_frame.grid(column=0, row=1, padx=20, pady=20, sticky="nsew")

label_file_explorer = ttk.Label(content_frame,
                               text="Select a file to explore",
                               style='TLabel',
                               foreground="#3498db")
label_file_explorer.grid(column=0, row=0, pady=(0, 20))

button_frame = ttk.Frame(content_frame)
button_frame.grid(column=0, row=1, pady=10)

button_explore = ttk.Button(button_frame,
                           text="Browse Files",
                           command=browseFiles,
                           style='Accent.TButton')
button_explore.grid(column=0, row=0, padx=5)

button_exit = ttk.Button(button_frame,
                        text="Exit",
                        command=exit,
                        style='TButton')
button_exit.grid(column=1, row=0, padx=5)


window.columnconfigure(0, weight=1)
window.rowconfigure(1, weight=1)
content_frame.columnconfigure(0, weight=1)

separator = ttk.Separator(window, orient='horizontal')
separator.grid(column=0, row=2, sticky="ew", pady=5)

status_bar = ttk.Label(window,
                      text="Ready",
                      relief='sunken',
                      anchor='w',
                      font=('Helvetica', 8))
status_bar.grid(column=0, row=3, sticky="ew", padx=5, pady=5)

style.configure('Accent.TButton',
               foreground='white',
               background='#3498db',
               font=('Helvetica', 10, 'bold'))
style.map('Accent.TButton',
         foreground=[('pressed', 'white'), ('active', 'white')],
         background=[('pressed', '#2980b9'), ('active', '#2980b9')])

window.mainloop()


Selected file path: /home/raulpuscas/ai/test1.ogg


/home/raulpuscas/ai/proj/sklearn-env/lib64/python3.13/site-packages/whisper/transcribe.py:130: UserWarning: Performing inference on CPU when CUDA is available
  warnings.warn("Performing inference on CPU when CUDA is available")
/home/raulpuscas/ai/proj/sklearn-env/lib64/python3.13/site-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Diagnoses:
- Aortic stenosis

Findings:
- Aorta la inel - 10mm
- Aorta la sinusuri var salva - 15mm
- Aorta ascendenta - 14mm
- Ventriculul drept - 10mm
- Atriu sting - 12mm

Measurements:
- Aorta inel - 10mm
- Aorta sinusuri - 15mm
- Aorta ascendenta - 14mm

Clinical Impressions:

Anatomical References:
- Aorta
- Ventriculul drept
- Atriul stang

Procedures:


/tmp/ipykernel_20010/1818595910.py:43: DeprecationWarning: This process (pid=20010) is multi-threaded, use of fork() may lead to deadlocks in the child.
  pid = os.fork()


This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
(/home/raulpuscas/ai/test1.tex
LaTeX2e <2022-11-01> patch level 1
L3 programming layer <2023-02-22>
(/usr/share/texlive/texmf-dist/tex/latex/base/article.cls
Document Class: article 2022/07/02 v1.4n Standard LaTeX document class
(/usr/share/texlive/texmf-dist/tex/latex/base/size10.clo))
(/usr/share/texlive/texmf-dist/tex/generic/babel/babel.sty
(/usr/share/texlive/texmf-dist/tex/generic/babel/txtbabel.def)
(/usr/share/texlive/texmf-dist/tex/generic/babel-english/english.ldf)
(/usr/share/texlive/texmf-dist/tex/generic/babel-romanian/romanian.ldf))
(/usr/share/texlive/texmf-dist/tex/generic/babel/locale/ro/babel-romanian.tex)
(/usr/share/texlive/texmf-dist/tex/generic/babel/locale/en/babel-english.tex)
(/usr/share/texlive/texmf-dist/tex/latex/graphics/graphicx.sty
(/usr/share/texlive/texmf-dist/tex/latex/graphics/keyval.sty)
(/usr/share/texlive/